In [ ]:
%load_ext autoreload
%autoreload 2

# Ti-64 beta-phase grain growth phase-field simulations

This notebook invokes code to set up a grain-growth CIPHER simulation for a Ti-64 beta-phase material.

In [ ]:
import plotly.express as px
import numpy as np

from cipher_input import (
    CIPHERInput,
    MaterialDefinition,
    PhaseTypeDefinition,
    InterfaceDefinition,
)
from utilities import read_shockley

### Assigning GB energies according to a random texture and simple misorientation-GB-energy relation

In [ ]:
RS_params = {
    'E_max': 1.2,
    'theta_max': 50,
    'degrees': True,
}
theta_deg = np.linspace(0, 100)
energy = read_shockley(theta_deg, **RS_params)
fig = px.line(
    x=theta_deg,
    y=energy,
    labels={"x": "Misorientation angle /deg.", "y": "GB energy / Jm^-2"},
    title='Read-Shockley relationship for LAGBs',
    width=600,
)
fig

In [ ]:
# Define the material properties:
materials = [    
    MaterialDefinition(
        name="Ti-beta",
        phase_types=[
            PhaseTypeDefinition(type_label='sub-grain-matrix'),
            PhaseTypeDefinition(type_label='nuclei'),
        ],
        properties={
            "chemicalenergy": "none",
            "molarvolume": 1e-5,
            "temperature0": 500.0,
        },
    ),
]

# Define the interfaces:
interfaces = [
    InterfaceDefinition(
        phase_types=("Ti-beta-sub-grain-matrix", "Ti-beta-sub-grain-matrix"),
        properties={
            "energy": {"e0": 5.0e+8},
            "mobility": {"m0": 1.0e-11},
        },
    ),
    InterfaceDefinition(
        phase_types=("Ti-beta-sub-grain-matrix", "Ti-beta-nuclei"),
        properties={
            "energy": {"e0": 5.0e+8},
            "mobility": {"m0": 1.0e-11},
        },
    ),
    InterfaceDefinition(
        phase_types=("Ti-beta-nuclei", "Ti-beta-nuclei"),
        properties={
            "energy": {"e0": 5.0e+8},
            "mobility": {"m0": 1.0e-11},
        },
    ),
]

# We also need provide a `phase_type_map` to state which Dream3D phases correspond to which
# CIPHER phase types:
inp = CIPHERInput.from_dream3D(
    path="example_data/dream3d/beta-grain-growth/synthetic_d3d.dream3d",
    materials=materials,
    interfaces=interfaces,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters={
        'abstol': 0.0001,
        'amrinterval': 25,
        'initblocksize': [1, 1, 1],
        'initcoarsen': 6,
        'initrefine': 7,
        'interfacewidth': 4,
        'interpolation': 'cubic',
        'maxnrefine': 7,
        'minnrefine': 0,
        'outfile': 'out',
        'outputfreq': 100,
        'petscoptions': '-ts_adapt_monitor -ts_rk_type 2a',
        'random_seed': 1579993586,
        'reltol': 0.0001,
        'time': 100000000,
    },
    phase_type_map={
        'Sub-grain matrix': 'Ti-beta-sub-grain-matrix',
        'Nuclei': 'Ti-beta-nuclei',
    }
)

In [ ]:
viz_slice_idx = 10
px.imshow(inp.geometry.voxel_phase[viz_slice_idx])

In [ ]:
inp.geometry.phase_orientation

In [ ]:
px.imshow(inp.geometry.voxel_material[viz_slice_idx])

In [ ]:
px.imshow(inp.geometry.voxel_phase_type[viz_slice_idx])

In [ ]:
misori = inp.geometry.get_misorientation_matrix(degrees=True)
px.imshow(misori)

In [ ]:
E_GB = read_shockley(misori, **RS_params)
px.imshow(E_GB)

In [ ]:
# show misorientation/GB-energies on Read-Shockley plot:
fig = px.line(
    x=theta_deg,
    y=energy,
    labels={"x": "Misorientation angle /deg.", "y": "GB energy / Jm^-2"},
    title='Read-Shockley relationship for LAGBs',
    width=600,
)
misori_sample = misori.flatten()
E_GB_sample = E_GB.flatten()
if inp.geometry.num_phases > 20:
    sample_idx = np.random.choice(misori_sample.size, size=400, replace=False)
    misori_sample = misori_sample[sample_idx]
    E_GB_sample = E_GB_sample[sample_idx]

fig.add_scatter(x=misori_sample, y=E_GB_sample, mode='markers')
fig

In [ ]:
for i in inp.interface_names:
    print(f'\nInterface: {i}')
    inp.apply_interface_property(
        base_interface_name=i,
        property_name=('energy', 'e0'),
        property_values=E_GB * 1e9,
        additional_metadata={'misorientation': misori},
        bin_edges=np.linspace(0, RS_params['E_max'] * 1e8, num=10),
    )

In [ ]:
inp.interface_names

In [ ]:
inp.write_yaml('ti64-beta-gg.yaml')

In [ ]:
px.imshow(inp.geometry.get_interface_idx()[viz_slice_idx])

In [ ]:
inp.geometry.show()